In [2]:
import acoular as ac # type: ignore
import numpy as np # type: ignore
from config import uma16_index, ConfigUMA
import acoupipe.sampler as sp # type: ignore
from scipy.stats import uniform # type: ignore
import tensorflow as tf # type: ignore
from modelsdfg.transformer.config import ConfigBase# type: ignore
from scipy.signal import welch # type: ignore
import h5py

/home/rabea/micromamba/envs/ba/lib/python3.11/site-packages/acoular/h5files.py:5: UserWarning: We detected that Numpy is already loaded and uses OpenBLAS. Because this conflicts with Numba parallel execution, we disable parallel execution for now and processing might be slower. To speed up, either import Numpy after Acoular or set environment variable OPENBLAS_NUM_THREADS=1 before start of the program.
  from .configuration import config
2024-08-19 10:57:03.845528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 10:57:03.845555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 10:57:03.846306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attemptin

In [4]:
folder = "uma16_example_recordings/"
hdf5_filename = folder + "450_and_700_1s_h5py.h5"

with h5py.File(hdf5_filename, 'r') as hdf5_file:
    signal = hdf5_file['data'][:]
    sample_freq = hdf5_file.attrs['sample_freq']
    channels = hdf5_file.attrs['channels']

print("Loaded signal shape:", signal.shape)
print("Sample freq:", sample_freq)
print("Number of channels:", channels)

fs = sample_freq #Abtastrate
t = np.arange(signal.shape[0]) / fs
df = 1/fs #Zeitabstand zwischen den Samples

Loaded signal shape: (44100, 16)
Sample freq: 44100.0
Number of channels: 16


In [12]:
def calculate_csm(signal):
    num_of_samples, num_of_channels = signal.shape
    fft_signal = np.fft.fft(signal, axis=0)
    csm = np.einsum('ij,ik->jk', fft_signal, np.conjugate(fft_signal)) / num_of_samples
    
    return csm

In [3]:
def calculate_csm_welch(signal, fs, nperseg=None, noverlap=None):
    freqs, Pxx = welch(signal, fs=fs, nperseg=nperseg, noverlap=noverlap, axis=0)
    csm = np.einsum('ij,ik->ijk', Pxx, np.conjugate(Pxx))
    return csm, freqs

fs = 44100  # Beispielhafte Abtastrate
nperseg = 1024  # Anzahl der Samples pro Segment
noverlap = 512  # Anzahl der überlappenden Samples



In [4]:
model_dir = "/home/rabea/Documents/Bachelorarbeit/models/EigmodeTransformer_learning_rate0.00025_epochs500_2024-04-10_19-09"
model_config_path = model_dir + "/config.toml"
ckpt_path = model_dir + '/ckpt/best_ckpt/0441-0.83.keras'

uma_config = ConfigUMA()
mic_index = uma16_index()
        
dev = ac.SoundDeviceSamplesGenerator(device=mic_index, numchannels=16)
recording_time = 1
dev.numsamples = int(recording_time * dev.sample_freq)
t = np.arange(dev.numsamples) / dev.sample_freq
        
model_config = ConfigBase.from_toml(model_config_path)
pipeline = model_config.datasets[1].pipeline.create_instance()
ref_mic_index = model_config.datasets[0].pipeline.args['ref_mic_index']
model_config.datasets[1].validation.cache=False
model = tf.keras.models.load_model(ckpt_path)



UMA-16 device: nanoSHARC micArray16 UAC2.0: USB Audio (hw:3,0) at index 5



2024-08-19 09:54:36.897049: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-19 09:54:36.958247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-19 09:54:36.958624: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
def prediction(signal):
    #csm = calculate_csm(signal)
    csm, freqs = calculate_csm_welch(signal, fs, nperseg, noverlap)
    eigmode = model.preprocessing(csm[np.newaxis]).numpy()
    strength_pred, loc_pred, noise_pred = model.predict(eigmode)
    strength_pred = strength_pred.squeeze()
    csm_norm = csm[ref_mic_index, ref_mic_index]
    csm = csm / csm_norm
    strength_pred *= np.real(csm_norm)
    loc_pred = pipeline.recover_loc(loc_pred.squeeze(), aperture=uma_config.mics.aperture)
    return strength_pred, loc_pred, noise_pred

In [6]:
for i in range(3):
    signal = ac.tools.return_result(dev, num=256)
    strength_pred, loc_pred, noise_pred = prediction(signal)
    print("num of sources", len(loc_pred[0]))
    

{'name': 'nanoSHARC micArray16 UAC2.0: USB Audio (hw:3,0)', 'index': 5, 'hostapi': 0, 'max_input_channels': 16, 'max_output_channels': 0, 'default_low_input_latency': 0.008684807256235827, 'default_low_output_latency': -1.0, 'default_high_input_latency': 0.034829931972789115, 'default_high_output_latency': -1.0, 'default_samplerate': 44100.0} 44100.0


2024-08-19 09:54:55.260387: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x5f0dac880840
2024-08-19 09:54:55.362719: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Exception encountered when calling layer 'eigmode_preprocessing' (type EigmodePreprocessing).

{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:Mul] name: 

Call arguments received by layer 'eigmode_preprocessing' (type EigmodePreprocessing):
  • csm=tf.Tensor(shape=(1, 513, 16, 16), dtype=float32)